<a href="https://colab.research.google.com/github/rostro36/Partisan-Responses/blob/master/06a_RoBERTa_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/Partisan-Responses-master')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Install all necessary libraries

In [ ]:
%%writefile setups.sh

git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Overwriting setups.sh


In [ ]:
!sh setups.sh

Cloning into 'apex'...
remote: Enumerating objects: 7431, done.
remote: Total 7431 (delta 0), reused 0 (delta 0), pack-reused 7431
Receiving objects: 100% (7431/7431), 13.90 MiB | 8.46 MiB/s, done.
Resolving deltas: 100% (5024/5024), done.
Checking out files: 100% (290/290), done.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-94632y8f
Created temporary directory: /tmp/pip-req-tracker-x4wfskw3
Created requirements tracker '/tmp/pip-req-tracker-x4wfskw3'
Created temporary directory: /tmp/pip-install-n3e7xv_b
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-7hn7sneq
  Added file:///content/drive/My%20Drive/Partisan-Responses-master/apex to build tracker '/tmp/pip-req-tracker-x4wfskw3'
    Running setup.py (path:/tmp/pip-re

In [ ]:
#!pip install -r requirements.txt
!pip install simpletransformers

     |████████████████████████████████| 215kB 6.9MB/s 
     |████████████████████████████████| 1.4MB 23.3MB/s 
     |████████████████████████████████| 7.2MB 40.2MB/s 
     |████████████████████████████████| 71kB 9.0MB/s 
     |████████████████████████████████| 3.0MB 34.8MB/s 
     |████████████████████████████████| 317kB 27.0MB/s 
     |████████████████████████████████| 778kB 40.4MB/s 
     |████████████████████████████████| 122kB 29.8MB/s 
     |████████████████████████████████| 163kB 47.8MB/s 
     |████████████████████████████████| 102kB 10.9MB/s 
     |████████████████████████████████| 102kB 11.3MB/s 
     |████████████████████████████████| 112kB 28.2MB/s 
     |████████████████████████████████| 4.4MB 36.9MB/s 
     |████████████████████████████████| 890kB 34.4MB/s 
     |████████████████████████████████| 1.1MB 33.7MB/s 
     |████████████████████████████████| 71kB 9.8MB/s 
     |████████████████████████████████| 71kB 8.8MB/s 
     |████████████████████████████████| 122kB 38.5MB/s 

Import all used libraries

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
from sklearn.model_selection import train_test_split

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


Read data

In [ ]:
speech = pd.read_pickle("all_speech_sentence_filtered.pkl")

sp = speech['Questions']
party = speech['Party'].apply(lambda x: 1 if x == 'R' else 0)

df = pd.DataFrame({
    'text': sp,
    'labels': party
})
df=df.dropna(axis=0, how='any')
df=df[df.iloc[:,0].apply(lambda x: len(x.split())<1000)]
df=df.sample(frac=0.3, random_state=36)
train_df, eval_df = train_test_split(df, test_size=0.2)

Delete data to reduce RAM and VRAM footprint

In [ ]:
del speech
del sp
del party
del df

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)
import gc
gc.collect(generation=2)

0

Train model

In [ ]:
model_args = ClassificationArgs(num_train_epochs=1, fp16=False, sliding_window=True, learning_rate=4e-6, save_eval_checkpoints= False,save_steps=50000)
model = ClassificationModel(
    "roberta", "roberta-base", args=model_args
)
model.train_model(train_df, output_dir='roberta')

INFO:filelock:Lock 139889933873672 acquired on /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690.lock


INFO:filelock:Lock 139889933873672 released on /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690.lock
INFO:filelock:Lock 139889933883152 acquired on /root/.cache/torch/transformers/80b4a484eddeb259bec2f06a6f2f05d90934111628e0e1c09a33bd4a121358e1.49b88ba7ec2c26a7558dda98ca3884c3b80fa31cf43a1b1f23aef3ff81ba344e.lock


INFO:filelock:Lock 139889933883152 released on /root/.cache/torch/transformers/80b4a484eddeb259bec2f06a6f2f05d90934111628e0e1c09a33bd4a121358e1.49b88ba7ec2c26a7558dda98ca3884c3b80fa31cf43a1b1f23aef3ff81ba344e.lock


- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:filelock:Lock 139889933875632 acquired on /root/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b.lock


INFO:filelock:Lock 139889933875632 released on /root/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b.lock


...

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


INFO:simpletransformers.classification.classification_model: 298570 features created from 62381 samples.


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to roberta.


Evaluate model

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


INFO:simpletransformers.classification.classification_model: 15596 features created from 15596 samples.


INFO:simpletransformers.classification.classification_model:{'mcc': 0.4189201760155458, 'tp': 3544, 'tn': 7519, 'fp': 1017, 'fn': 3516, 'eval_loss': 0.5784562712215089}


In [ ]:
result

{'eval_loss': 0.5784562712215089,
 'fn': 3516,
 'fp': 1017,
 'mcc': 0.4189201760155458,
 'tn': 7519,
 'tp': 3544}